In [11]:
#1. 사용환경 준비
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ") # API 키 입력


In [12]:
#2. 모델 로드하기 
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [38]:
#3. 문서 로드하기
from langchain.document_loaders import CSVLoader

# PDF 파일 로드. 파일의 경로 입력
loader = CSVLoader("sum_sum_Cultural2.csv",encoding='UTF8')
loader2 = CSVLoader("place.csv")

# 페이지 별 문서 로드
Treasures = loader.load()
Travel = loader2.load()
print(Travel[:1])

RuntimeError: Error loading place.csv

In [27]:
#4. 문서 청크로 나누기(CharacterTextSplitter)
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document



# 텍스트 청크 분할기 설정 (문단 기준 분할)
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(Treasures) # 문서를 청크로 분할
splits2 = text_splitter.split_documents(Travel)
print(splits2[1]) # 상위 10개만 출력

page_content='title: 원대리 자작나무 숲 (속삭이는 자작나무 숲)
address: 강원특별자치도 인제군 인제읍 자작나무숲길 760
telNo: 자작나무숲 안내소 033-463-0044' metadata={'source': 'place.csv', 'row': 1}


# CharacterTextSplitter 청킹 방식 및 매개변수 설명

## 청킹 방식
`CharacterTextSplitter`는 텍스트를 지정된 기준에 따라 나누어 작은 조각(청크)으로 만드는 기능을 제공한다. 이 방식은 대량의 텍스트를 처리할 때 유용합니다.

## 매개변수 설명

1. **separator**: 
   - **설명**: 문서를 나눌 기준으로 사용할 문자열을 설정한다는 뜻이다.

2. **chunk_size**:
   - **설명**: 각 청크의 최대 크기를 설정한다. 청크는 이 크기를 초과하지 않는다.

3. **chunk_overlap**:
   - **설명**: 청크 간의 겹치는 글자 수를 설정한다. 문맥을 유지하는 데 도움이 된다.


4. **length_function**:
   - **설명**: 청크의 길이를 계산하는 함수이다. 일반적으로 문자열 길이를 계산하는 `len` 함수를 사용한다.


5. **is_separator_regex**:
   - **설명**: 구분자가 정규 표현식인지 여부를 설정한다. `False`로 설정하면 일반 문자열로 처리된다.


## 사용 목적
이 과정을 통해 대량의 텍스트를 효과적으로 처리하고, AI 모델이 이해할 수 있는 형태로 변환하여 정보 추출 및 분석을 용이하게 합니다.

In [28]:
#5 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [ ]:
#6. 벡터 스토어 생성
import faiss
from langchain_community.vectorstores import FAISS

# 문서에서 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
vectorstore2 = FAISS.from_documents(documents=splits2, embedding=embeddings)


In [ ]:
#7. FAISS를 Retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [31]:
#8. 프롬프트 템플릿을 정의하라
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [ ]:
#9. LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성하라. 
from langchain.chains import LLMChain

class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_Treasures = self.retriever.get_relevant_documents(query)
        return response_Treasures
#텍스트 생성 체인 생성
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

## RAG(생성형 응답 생성)의 필요성

RAG는 정보 검색과 생성 모델을 결합하여 사용자 질문에 대한 보다 정확하고 관련성 높은 답변을 생성하는 기술이다. RAG의 필요성은 다음과 같은 이유로 설명할 수 있다.

1. **정보의 동적 접근**: RAG는 최신 정보를 실시간으로 검색할 수 있어, 사용자가 요청하는 질문에 대해 항상 최신의 정확한 정보를 제공할 수 있다.

2. **맥락 이해**: 사용자 질문에 대한 답변을 생성할 때, RAG는 관련된 맥락을 기반으로 하여 보다 깊이 있는 이해를 제공합니다. 이는 단순한 정답 제공을 넘어, 질문의 의도를 파악하여 적절한 답변을 생성하는 데 기여한다.

3. **대량의 데이터 처리**: RAG는 대량의 문서와 정보를 처리할 수 있는 능력을 가지고 있어, 사용자가 원하는 다양한 주제에 대해 신속하게 대응할 수 있다.

4. **개선된 사용자 경험**: RAG를 통해 제공되는 답변은 보다 개인화되고 관련성이 높아, 사용자 경험을 개선할 수 있다. 이는 사용자 만족도를 높이고, 반복적인 질문을 줄이는 데 도움이 된다.

결론적으로, RAG는 정보 검색과 생성 모델의 장점을 결합하여, 더 나은 품질의 질문 응답 시스템을 구축하는 데 필수적이다.

In [ ]:
#10. 질문에 응답하는 챗봇을 구동하여 질문해라. 

# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    if query == "종료": # 종료 입력 시 챗봇 종료
        break
    
    # 1. Retriever로 관련 문서 검색
    response_Treasures = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_Treasures,
        "question": query
    })
    
    

    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n질문:",query)
    print("\n답변:")
    print(response.content) # 답변 출력

KeyError: 'context_treasures'